In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Car_Price_Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Car_Price_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    random_state: 0
    target_column: str

In [6]:
from carpriceproject.constants import *
from carpriceproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            random_state= 0,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from carpriceproject import logger
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import joblib

from carpriceproject.components.data_transformation import DataTransformation

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        transformer = DataTransformation(config=self.config)
        train_data = transformer.preprocess_data(train_data)
        test_data = pd.read_csv(self.config.test_data_path)
        test_data = transformer.preprocess_data(test_data)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]



        # Initialize and train RandomForestRegressor
        model = RandomForestRegressor(random_state=0)
        model.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        print(f"Model trained and saved to {os.path.join(self.config.root_dir, self.config.model_name)}")


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-09 11:20:02,519: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-09 11:20:02,533: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 11:20:02,535: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 11:20:02,536: INFO: common: created directory at: artifacts]
[2024-07-09 11:20:02,537: INFO: common: created directory at: artifacts/model_trainer]


C:\Users\HP\Car_Price_Prediction\src\carpriceproject\components\data_transformation.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['name'].replace(brand_mapping, inplace=True)
C:\Users\HP\Car_Price_Prediction\src\carpriceproject\components\data_transformation.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent

Model trained and saved to artifacts/model_trainer\model.joblib
